In [ ]:
!pip install --upgrade --no-cache-dir gdown
!pip install diffusers
!pip install datasets
!pip install peft
!pip install multilingual-clip torch

In [ ]:
import gdown
url = "https://drive.google.com/uc?id=19-OXon37j72Zne8DKDQTPID-sd1VVreL"
output = "/content/code.zip"  # Specify the output path and file name

gdown.download(url, output, quiet=False)

In [ ]:
import zipfile
import os

zip_file_path = '/content/code.zip'
extract_folder_path = '/content/'

# Check if the zip file exists
if os.path.exists(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder_path)
else:
    print("The specified zip file does not exist.")

In [ ]:
from image_utils import imageToTensor

In [ ]:
from datasets import load_dataset
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
class CustomCartoonBlipCaptionsDataset(Dataset):
    def __init__(self, split='train', transform=None, max_samples=None):
        # Load the dataset
        # self.dataset = load_dataset("Norod78/cartoon-blip-captions", split=split)
        self.dataset = load_dataset("Norod78/simpsons-blip-captions", split=split)
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        # Get image path and caption from the dataset
        image = self.dataset[idx]["image"]
        caption = self.dataset[idx]["text"]
        # Apply transformations if provided
        if self.transform:
            image = image.convert('RGB')
            image = self.transform(image)
        return image, caption

# Define transformations to apply to the images
transform = transforms.Compose([
         transforms.Resize(512, interpolation=transforms.InterpolationMode.BILINEAR),
          transforms.CenterCrop(512) ,
          transforms.ToTensor(),
          transforms.Normalize([0.5], [0.5]),
])

# Limit the dataset to 2000 samples
max_samples = None

# Create an instance of the custom dataset
custom_dataset = CustomCartoonBlipCaptionsDataset(split='train', transform=transform, max_samples=max_samples)

# Access a sample from the dataset
sample_image, sample_caption = custom_dataset[0]
print(sample_image.shape)  # Print the shape of the image tensor
print(sample_caption)      # Print the caption associated with the image

Generating train split:   0%|          | 0/755 [00:00<?, ? examples/s]

torch.Size([3, 512, 512])
simpsons characters in an illustration, The Simpsons


In [ ]:
from models import Diffusion

In [ ]:
import torch
device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [ ]:
d = Diffusion(device , "DEIS")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
d.addLora()

In [ ]:
d.train(custom_dataset , createPickle = True ,learning_rate =1e-04 ,batch_size=7,epochs=50 )

epoch 0 loss 0.1118375874572882
epoch 1 loss 0.12521910727782934
epoch 2 loss 0.11571081663930306
epoch 3 loss 0.10717491543403378
epoch 4 loss 0.11812762450426817
epoch 5 loss 0.11695884614837943
epoch 6 loss 0.11825141079585862
epoch 7 loss 0.11734366283700284
epoch 8 loss 0.11998715738041534
epoch 9 loss 0.10681022269030412
epoch 10 loss 0.11000993761613413
epoch 11 loss 0.1220473143686023
epoch 12 loss 0.11071030071212186
epoch 13 loss 0.11339031887689123
epoch 14 loss 0.11581992513189714
epoch 15 loss 0.11742836652599552
epoch 16 loss 0.11688968534063962
epoch 17 loss 0.11986695919875745
epoch 18 loss 0.10938339175104543
epoch 19 loss 0.12336826819443593
epoch 20 loss 0.11554496831915996
epoch 21 loss 0.119779908546695
epoch 22 loss 0.10825946909823904
epoch 23 loss 0.11282301604471824
epoch 24 loss 0.11046195820111919
epoch 25 loss 0.11629920940915192
epoch 26 loss 0.12054668941224615
epoch 27 loss 0.10998035274032089
epoch 28 loss 0.11243043822684774
epoch 29 loss 0.114922475587

In [ ]:
trainable_params = {name: param for name, param in d.unet.model.named_parameters() if param.requires_grad}
torch.save(trainable_params, '/content/trainable_params_layer1.pth')

In [ ]:
from google.colab import files
files.download('/content/trainable_params_layer1.pth')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>